### **Data Reading**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *


In [0]:
df= spark.read.format("parquet")\
  .load("abfss://bronze@databricksete80.dfs.core.windows.net/orders")

In [0]:
display(df)

In [0]:
df.printSchema()

In [0]:
df1 = df.withColumnRenamed("_rescued_data","rescued_data")

In [0]:
df1.display()

In [0]:
df1 = df1.drop("rescued_data")

In [0]:
df1.display()

In [0]:
df1 = df1.withColumn("order_date",to_timestamp(col("order_date")))
df1.display()

In [0]:
df1 = df1.withColumn("Year", year(col("order_date")))
df1.display()

In [0]:
df2 = df1.withColumn("flag", dense_rank().over(Window.partitionBy("Year").orderBy(desc("total_amount"))))
df2.display()

In [0]:
df2 = df2.withColumn("row_flag", row_number().over(Window.partitionBy("Year").orderBy(desc("total_amount"))))
df2.display()

### **Classes - OOP**

In [0]:
class windows:


    def dense_rank(self,df):
        df_dense_rank = df.withColumn("flag", dense_rank().over(Window.partitionBy("Year").orderBy(desc("total_amount"))))
        return df_dense_rank

    def rank(self,df):
        df_rank = df.withColumn("row_flag", row_number().over(Window.partitionBy("Year").orderBy(desc("total_amount"))))
        return df_rank

    def row_number(self,df):
        df_row_number = df.withColumn("row_flag", row_number().over(Window.partitionBy("Year").orderBy(desc("total_amount"))))
        return df_row_number

In [0]:
df_new = df1

In [0]:
df_new.display()

In [0]:
obj = windows()

In [0]:
obj.dense_rank(df_new).display()
obj.rank(df_new).display()
obj.row_number(df_new).display()


### **Data Writing**

In [0]:
df1.write.format("delta").mode("append").save("abfss://silver@databricksete80.dfs.core.windows.net/orders")